### DB 접근

### 구 리스트 찾기

In [139]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import UnexpectedAlertPresentException, NoSuchElementException, StaleElementReferenceException
import time
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import UnexpectedAlertPresentException
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import random

gu_list = []

gu_combo_box = driver.find_element(By.XPATH, '//*[@id="selectSigungu"]')
gu_panel = gu_combo_box.find_elements(By.TAG_NAME, 'option')[1:]  # 구 이름 리스트

for idx, each_gu in enumerate(gu_panel):
    gu_code = each_gu.get_attribute('value')  # 구별 고유 코드
    gu_name = each_gu.text
    gu_list.append([gu_name,gu_code])

gu_list[17:18]

[['송파구', '11710']]

### 동 리스트 찾기

In [142]:
dong_list = []  # 구가 초기화 될 때마다 dong_list 도 초기화 되야 함.
dong_combo_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="selectBjdong"]')))
select_dong = Select(dong_combo_box) 
dong_panel = dong_combo_box.find_elements(By.TAG_NAME, 'option')[1:]
# 해당 구의 동 선택하기
dong_list = []  # 구가 초기화 될 때마다 dong_list 도 초기화 되야 함.

# 우선 현재 구의 동 html 리스트를 생성하고
for idx, each_dong in enumerate(dong_panel):
    dong_code = each_dong.get_attribute('value')  # 동별 고유 코드
    dong_name = each_dong.text  # 동 이름
    dong_list.append([dong_name, dong_code])

dong_list[0:1]

[['가락동', '10700']]

# 송파구 가락동 연도별 아파트 매매 가격 크롤링 코드

In [ ]:
import re
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import UnexpectedAlertPresentException, NoSuchElementException, StaleElementReferenceException
import time
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import UnexpectedAlertPresentException
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import random

sql_insert_query = """
INSERT INTO APARTMENT_PRICE_GARAK (YEAR, MONTH, GU, DONG, APARTMENT, SIZE_m2, SIZE_평, PRICE)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

options = webdriver.ChromeOptions()
options.add_argument(f'user-agent={generate_user_agent()}')  # 무작위 User-Agent 설정


url = 'https://land.seoul.go.kr/land/rtms/rtmsApartment.do'  # 부동산 통계정보 사이트 url
small_driver = webdriver.Chrome(service=Service('/home//ws/eda/driver/chromedriver'),
                          options=options)

small_driver.get(url)

small_driver.maximize_window()  # 전체화면 크기로 지정

driver = small_driver

# 날짜 리스트 생성하기
# 2024년 08월 부터 2014년 01 월 까지 연도 기준 역순으로
start = 20240101
end = 20240931

start_year_list = []
end_year_list = []

for i in range(11):
    start_year_list.append(start)
    start -= 10000

#print('=' * 50)

for j in range(11):
    end_year_list.append(end)
    if end == 20240931:
        end -= 9700
    else:
        end -= 10000

start_date_box = driver.find_element(By.XPATH, '//*[@id="changeBgnde"]')  # 시작일 콤보박스
end_date_box = driver.find_element(By.XPATH, '//*[@id="changeEndde"]')  # 종료일 콤보박스

# 구 리스트 구하기
gu_list = []

gu_combo_box = driver.find_element(By.XPATH, '//*[@id="selectSigungu"]')

# gu_combo_box를 Select 클래스로 감싸기
select_gu = Select(gu_combo_box)

# 각 구 이름 리스트들 우선 구해놓고
gu_panel = gu_combo_box.find_elements(By.TAG_NAME, 'option')[1:]  # 구 이름 리스트

# 각 구에 해당하는 'value' 값 구해놓기
for idx, each_gu in enumerate(gu_panel):
    gu_code = each_gu.get_attribute('value')  # 구별 고유 코드
    gu_name = each_gu.text
    gu_list.append([gu_name, gu_code])

# 구 선택하기 (송파구 선택)----------------------------------------------------------------------------------
gu_code = gu_list[17][1]
gu_name = gu_list[17][0]
select_gu.select_by_value(gu_code)  # 각 구를 선택해준다.
time.sleep(0.2)

# 구를 선택했다면 이제 동 선택하기 ----------------------------------------------------------------
# 동 콤보박스에서
#dong_list = []  # 구가 초기화 될 때마다 dong_list 도 초기화 되야 함.
dong_combo_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="selectBjdong"]')))
select_dong = Select(dong_combo_box) 
dong_panel = dong_combo_box.find_elements(By.TAG_NAME, 'option')[1:]
# 해당 구의 동 선택하기
dong_list = []  # 구가 초기화 될 때마다 dong_list 도 초기화 되야 함.

# 우선 현재 구의 동 html 리스트를 생성하고
for idx, each_dong in enumerate(dong_panel):
    dong_code = each_dong.get_attribute('value')  # 동별 고유 코드
    dong_name = each_dong.text  # 동 이름
    dong_list.append([dong_name, dong_code])

# 가락동 선택
code = dong_list[0][1]  # 동 코드
dong_name = dong_list[0][0]  # 동 이름
select_dong.select_by_value(code)  # 각 동 클릭하기

# 동까지 클릭했다면 날짜를 가져와야한다. ----------------------------------------------------------
# 날짜 연 단위로 내려가는 함수
for idx in range(len(start_year_list)):
    start_date_box = driver.find_element(By.XPATH, '//*[@id="changeBgnde"]')  # 시작일 콤보박스
    end_date_box = driver.find_element(By.XPATH, '//*[@id="changeEndde"]')  # 종료일 콤보박스

    start_date_box.send_keys(Keys.CONTROL, 'a', Keys.BACKSPACE)  # 시작 날짜 지우고
    start_date_box.send_keys(start_year_list[idx])  # 시작 날짜 보내기

    end_date_box.click()
    
    # 경고창이 뜬 경우 이를 처리하는 코드
    try:
        # 경고창이 나타나면 Alert 객체로 변환
        alert = Alert(driver)
        
        # 경고창 닫기 (확인 버튼 클릭)
        alert.accept()
        print("경고창이 닫혔습니다.")
    except:
        # 경고창이 없으면 예외 처리
        print("경고창이 없습니다.")

    end_date_box.send_keys(Keys.CONTROL, 'a', Keys.BACKSPACE)  # 끝나는 날짜 지우고
    end_date_box.send_keys(end_year_list[idx])  # 종료 날짜 보내기

    while True:
        # '검색' 클릭
        search_btn = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search"]')))
        search_btn.click()
        # 경고창이 뜬 경우 이를 처리하는 코드
        try:
            # 경고창이 나타나면 Alert 객체로 변환
            alert = Alert(driver)
            
            # 경고창 닫기 (확인 버튼 클릭)
            alert.accept()
            print("경고창이 닫혔습니다.")
        except:
            # 경고창이 없으면 넘어가고
            print("경고창이 없습니다.")
            break

    # 표 보일 때까지 스크롤
    element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'resultList2')))
    driver.execute_script("arguments[0].scrollIntoView();", element)
    print('스크롤 완료')

    while True:  # 표가 로딩될 때까지 계속 반복
        try:
            # 표가 보이면
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, 'resultList2')))

            # 현재 페이지 html 로 가져오기
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')  
            print('표 보임')

            # 필요한 정보들 가져오기 (아파트 이름, 전용 면적, 계약 년도, 계약 월, 거래 금액)
            info_cell = soup.find('tbody', id='resultList2')  # 현재 단지의 표
            info_row = info_cell.find_all('tr')  # 모든 행들을 info_row 에 저장

            last_valid_name = ''  # 이전 유효한 이름을 저장할 변수
            last_valid_size = ''  # 이전 유효한 전용 면적을 저장할 변수

            for idx, each_row in enumerate(info_row):  # 첫 번째 행 가져와서
                each_info = each_row.find_all('td')  # 현재 행의 모든 'td' 태그 찾기
                
                # 'td'가 존재하고, 'td'의 첫 번째 요소의 텍스트가 None이 아닌지 확인
                if each_info and each_info[0] is not None and each_info[0].get_text() is not None:
                    name = each_info[0].get_text().replace(u"\xa0", u"").strip()
                else:
                    name = ''  # 'td'가 없으면 빈 문자열 할당

                # name이 빈 문자열이라면, 이전 행의 이름을 가져오고, 여전히 공백이면 마지막으로 유효했던 이름 사용
                if name == '':
                    if idx > 0:  # 현재 행이 첫 번째가 아니면
                        for_name = info_row[idx - 1]
                        name_info = for_name.find_all('td') 
                        name = name_info[0].get_text().replace(u"\xa0", u"").strip()

                    if name == '':  # 이전 행의 name도 빈 문자열이면 마지막 유효한 name 사용
                        name = last_valid_name

                # 유효한 name을 업데이트
                if name != '':
                    last_valid_name = name

                # 준공 년도는 제외하고 아파트 이름만 추출
                real_name = name.split('[')[0]

                # 전용 면적 처리
                size = each_info[2].string.strip()  # 전용 면적 (m^2) 단위
                if size == '':  # 만약 전용면적이 빈칸이면
                    size = last_valid_size  # 이전의 유효한 size 값을 가져옴

                # 유효한 size를 업데이트
                if size != '':
                    last_valid_size = size

                # 계약 연도와 월 추출
                year = int('20' + each_info[3].string[:2])  # 연도
                month = int(each_info[3].string[3:5])  # 월
                convert_size = round(float(size) * 0.3025, 1)  # 평수로 변환
                
                # 거래 금액 처리
                price = int(each_info[4].string.replace(',', '')) * 10000

                # 결과 출력
                print(gu_name, dong_name, real_name, float(size), convert_size,year, month, price)
                cur.execute(sql_insert_query, (year, month, gu_name, dong_name, real_name, float(size), convert_size, price))
                remote.commit()
            break
        except:
            print('오류 발생!!! 속성 값을 다시 찾습니다!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print('=' * 50)




경고창이 없습니다.
경고창이 없습니다.
스크롤 완료
표 보임
오류 발생!!! 속성 값을 다시 찾습니다!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
표 보임
오류 발생!!! 속성 값을 다시 찾습니다!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
표 보임
오류 발생!!! 속성 값을 다시 찾습니다!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
표 보임
오류 발생!!! 속성 값을 다시 찾습니다!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
표 보임
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 1 1072000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 1 1045000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 1 1055000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 1 1060000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 1 1065000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 1 1155000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 1 1110000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 1 1095000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 2 1050000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 2 1070000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 2 1050000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 2 1100000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 3 1110000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 3 1075000000
송파구 가락동 가락(1차)쌍용아파트 59.92 18.1 2024 3 1060000000

In [1]:
cur.close()
remote.close()
driver.close()

NameError: name 'cur' is not defined